In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision
set_global_policy('mixed_float16')

# Set data path
data_path = # Enter file path here

# Load the data
data = np.load(data_path, mmap_mode='r')

# Calculate total number of samples
total_samples = 100 * 90  # 9000 samples (100 months, 10x9 grid)

# Create random indices for training and testing
np.random.seed(42)  # for reproducibility
all_indices = np.random.permutation(total_samples)
train_indices = all_indices[:7200]
test_indices = all_indices[7200:]

def generate_sample(index):
    month = index // 90 + 12  # Start from 12th month (0-based index)
    block = index % 90
    row = block // 9  # 0-9 for 10 rows
    col = block % 9  # 0-8 for 9 columns

    X = data[:, month-12:month, row, col]  # Shape: (4, 12, 64, 64, 3)
    y = data[0, month, row, col]  # Shape: (64, 64, 3)

    # Replace NaN values with 0
    X = np.nan_to_num(X, nan=0.0)
    y = np.nan_to_num(y, nan=0.0)

    return X, y

def create_dataset(indices, batch_size):
    def gen():
        for index in indices:
            yield generate_sample(index)

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(4, 12, 64, 64, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32)
        )
    )
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE).repeat()